## 1. Import Modules

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

## 2. Data Load & Segmentation

In [2]:
train_data, valid_data, test_data = tfds.load(
    name='imdb_reviews',
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

In [3]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [4]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## 3. Network Structure

사전 훈련된 텍스트 Embedding 모델

In [5]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

## 4. Training

In [9]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=valid_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 4s 69ms/step - loss: 1.1354 - accuracy: 0.4719 - val_loss: 0.7892 - val_accuracy: 0.4642
Epoch 2/20
30/30 [==============================] - 2s 67ms/step - loss: 0.7308 - accuracy: 0.5445 - val_loss: 0.6895 - val_accuracy: 0.5883
Epoch 3/20
30/30 [==============================] - 3s 67ms/step - loss: 0.6604 - accuracy: 0.6200 - val_loss: 0.6324 - val_accuracy: 0.6463
Epoch 4/20
30/30 [==============================] - 2s 68ms/step - loss: 0.6051 - accuracy: 0.6726 - val_loss: 0.5836 - val_accuracy: 0.6924
Epoch 5/20
30/30 [==============================] - 2s 66ms/step - loss: 0.5570 - accuracy: 0.7161 - val_loss: 0.5422 - val_accuracy: 0.7269
Epoch 6/20
30/30 [==============================] - 2s 67ms/step - loss: 0.5137 - accuracy: 0.7544 - val_loss: 0.5051 - val_accuracy: 0.7592
Epoch 7/20
30/30 [==============================] - 2s 66ms/step - loss: 0.4721 - accuracy: 0.7849 - val_loss: 0.4710 - val_accuracy: 0.7872
Epoch 8/20
30

## 5. Model Evaluation

In [10]:
results = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
    print("{0}: {1:.3f}".format(name, value))

49/49 - 2s - loss: 0.3199 - accuracy: 0.8637
loss: 0.320
accuracy: 0.864
